In [149]:
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import sklearn.preprocessing
import sklearn.metrics
from data_split_tune_utils import X_y_site_split

In [150]:
### read in train, val, and test
train = pd.read_csv('../data/trainV_ridgeImp.csv')
val = pd.read_csv('../data/valV_ridgeImp.csv')
test = pd.read_csv('../data/testV_ridgeImp.csv')

In [319]:
### split train, val, and test into x, y, and sites
train_x, train_y, train_sites = X_y_site_split(train, y_var_name='MonitorData', site_var_name='site')
val_x, val_y, val_sites = X_y_site_split(val, y_var_name='MonitorData', site_var_name='site')
test_x, test_y, test_sites = X_y_site_split(test, y_var_name='MonitorData', site_var_name='site')

In [228]:
#train_x = train_x.iloc[:, :40]

In [229]:
#train_x_std_all = train_x

In [153]:
### standardize all features
standardizer_all = sklearn.preprocessing.StandardScaler(with_mean = True, with_std = True)
train_x_std_all = standardizer_all.fit_transform(train_x)
val_x_std_all = standardizer_all.transform(val_x)
test_x_std_all = standardizer_all.transform(test_x)

In [120]:
### NN model architecture
class NN(torch.nn.Module):
    def __init__(self, input_size_full, hidden_size_full):
        super(NN, self).__init__()
        #self.linear_full = torch.nn.Linear(in_features=input_size_full, out_features=hidden_size_full, bias=True)
        #self.norm_full = torch.nn.BatchNorm1d(num_features = hidden_size_full)
        #self.relu_full = torch.nn.ReLU()
        self.linear_full2 = torch.nn.Linear(in_features=input_size_full, out_features=1, bias=True)
        
    def forward(self, input_full):
        #hidden_full = self.linear_full(input_full)
        #hidden_full = self.norm_full(hidden_full)
        #hidden_full = self.relu_full(hidden_full)
        output_full = self.linear_full2(input_full)

        return output_full

In [311]:
# NN parameters
input_size_full = train_x_std_all.shape[1]
#input_size_full = 1
hidden_size_full = 100

In [312]:
nn = torch.nn.Sequential()
nn.add_module('linear', torch.nn.Linear(in_features=input_size_full, out_features=1, bias=True))

In [313]:
# instantiate model
#nn = NN(input_size_full, hidden_size_full)

In [314]:
# Loss function
mse_loss = torch.nn.MSELoss(size_average=True)

In [315]:
# Optimizer
lr = 0.0001
weight_decay = 0.000001
optimizer = torch.optim.Adam(nn.parameters(), lr=lr, weight_decay=weight_decay)

In [316]:
num_epochs = 50
batch_size = 500

In [317]:
# get number of batches
if train_x_std_all.shape[0] % batch_size != 0:
    num_batches = int(np.floor(train_x_std_all.shape[0]/batch_size) + 1)
else:
    num_batches = int(train_x_std_all.shape[0]/batch_size)

In [318]:
for epoch in range(num_epochs):
    epoch_loss = 0
    pred_epoch = []
    y_epoch = []
    
    for batch in range(num_batches):
        # get x and y for this batch
        x = train_x_std_all[batch_size * batch:batch_size * (batch+1)].values
        y = train_y[batch_size * batch:batch_size * (batch+1)].values
        
        # get indices for monitor data and actual monitor data
        y_nan_ind = np.isnan(y)
        
        if np.sum(y_nan_ind) < len(y_nan_ind):
            x = Variable(torch.from_numpy(x[~y_nan_ind])).float()
            y = Variable(torch.from_numpy(y[~y_nan_ind])).float()

            # get model output
            pred_batch = nn(x)
            #pred_batch = pred_batch[torch.from_numpy(y_nan_ind.values.argsort()[:len(y_nan_ind.values) - np.sum(y_nan_ind.values)]).long()]

            # compute loss, backprop, and update parameters
            optimizer.zero_grad()
            loss_batch = mse_loss(pred_batch, y)
            loss_batch.backward()
            optimizer.step()

            # accumulate loss over epoch
            pred_epoch += list(pred_batch.data.numpy())
            y_epoch += list(y.data.numpy())
            epoch_loss += loss_batch.data[0]
            
    

    #print('Validation R^2 after epoch ' + str(epoch) + ': ' + str(r2(cnn, batch_size, val_x_std_stack_nonConst, val_x_std_tuple, val_y_tuple)))
    print(sklearn.metrics.r2_score(y_epoch, pred_epoch))
    print('Epoch loss after epoch ' + str(epoch) + ': ' + str(epoch_loss))
    

#print('Test R^2: ' + str(r2(cnn, batch_size, test_x_std_stack_nonConst, test_x_std_tuple, test_y_tuple)))

-27.3837367083
Epoch loss after epoch 0: 3511186.7910633087
-9.66310931106
Epoch loss after epoch 1: 1312121.5751695633
-3.79591421277
Epoch loss after epoch 2: 568538.1880731583
-0.709527868015
Epoch loss after epoch 3: 199373.35906243324
0.282875328478
Epoch loss after epoch 4: 83861.5897140503
0.464567856825
Epoch loss after epoch 5: 63016.57800912857
0.533406760999
Epoch loss after epoch 6: 55002.779361605644
0.580256867536
Epoch loss after epoch 7: 49510.05389910936
0.613094011813
Epoch loss after epoch 8: 45643.7243757844
0.636686384244
Epoch loss after epoch 9: 42851.08884048462
0.65420883478
Epoch loss after epoch 10: 40765.01810437441
0.667616159081
Epoch loss after epoch 11: 39159.97445091605
0.678097011326
Epoch loss after epoch 12: 37898.741636902094
0.686402558532
Epoch loss after epoch 13: 36894.38498274982
0.693038522144
Epoch loss after epoch 14: 36088.146144494414
0.698366209019
Epoch loss after epoch 15: 35437.86316435039
0.702656702032
Epoch loss after epoch 16: 3491